In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import urllib.parse

# ▶ CSV 파일 불러오기
df = pd.read_csv("C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/데이터/스캔들/ALL논란_Final.csv")
df["사건 날짜"] = pd.to_datetime(df["사건 날짜"]).dt.strftime("%Y.%m.%d")
name_date_df = df[["연예인 이름", "사건 날짜"]].drop_duplicates()

# ▶ 크롤링 함수
def get_news_count(name, date_str):
    query = urllib.parse.quote(name)
    date_param = date_str.replace(".", "")
    
    url = f"https://search.naver.com/search.naver?where=news&query={query}&sort=0&ds={date_str}&de={date_str}&nso=so:r,p:from{date_param}to{date_param},a:all"

    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(2)  # JS 로딩 대기

    try:
        result_text = driver.find_element(By.CLASS_NAME, "title_desc.all_my").text
        count = int(result_text.split("건")[0].split()[-1].replace(",", ""))
    except:
        count = 0

    driver.quit()
    return count

# ▶ 기사 수 수집 실행
counts = []
for _, row in name_date_df.iterrows():
    name, date = row["연예인 이름"], row["사건 날짜"]
    try:
        count = get_news_count(name, date)
        print(f"{name} / {date} → {count}건")
    except Exception as e:
        print(f"오류: {name} / {date} → {e}")
        count = None
    counts.append(count)

# ▶ 결과 저장
name_date_df["네이버 기사 수"] = counts
name_date_df.to_csv("연예인_네이버뉴스_기사수.csv", index=False)


지민 / 2022.04.25 → 0건
정국 / 2024.01.06 → 0건
정원 / 2021.11.18 → 0건
정원 / 2021.11.19 → 0건


Exception ignored in: <function Service.__del__ at 0x000001A222401620>
Traceback (most recent call last):
  File "c:\Users\chica\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdriver\common\service.py", line 200, in __del__
  File "c:\Users\chica\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdriver\common\service.py", line 157, in stop
  File "c:\Users\chica\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdriver\common\service.py", line 137, in send_remote_shutdown_command
  File "c:\Users\chica\AppData\Local\Programs\Python\Python312\Lib\urllib\request.py", line 215, in urlopen
  File "c:\Users\chica\AppData\Local\Programs\Python\Python312\Lib\urllib\request.py", line 515, in open
  File "c:\Users\chica\AppData\Local\Programs\Python\Python312\Lib\urllib\request.py", line 532, in _open
  File "c:\Users\chica\AppData\Local\Programs\Python\Python312\Lib\urllib\request.py", line 492, in _call_chain
  File "c:\Users\chica

KeyboardInterrupt: 

In [6]:
import pandas as pd
import urllib.parse
import requests
from bs4 import BeautifulSoup
from datetime import timedelta, datetime
import time

# 🔹 데이터 불러오기
df = pd.read_csv("C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/데이터/스캔들/ALL논란_Final.csv")
df["사건 날짜"] = pd.to_datetime(df["사건 날짜"])
name_date_df = df[["연예인 이름", "사건 날짜"]].drop_duplicates()

# 🔹 기사 수 수집 설정
pages = 5  # 5페이지면 약 50건까지 수집

results = []

# 🔁 연예인 사건별로 반복
for _, row in name_date_df.iterrows():
    name = row["연예인 이름"]
    date = row["사건 날짜"]

    start_str = date.strftime("%Y.%m.%d")
    end_str = (date + timedelta(days=0)).strftime("%Y.%m.%d")  # 사건 당일만 검색
    date_param = date.strftime("%Y%m%d")

    titles = []

    # 🔁 뉴스 제목 수집 (1~5페이지)
    for page in range(1, pages * 10, 10):
        url = (
            f"https://search.naver.com/search.naver"
            f"?where=news&query={urllib.parse.quote(name)}"
            f"&sort=0&ds={start_str}&de={end_str}&nso=so:r,p:from{date_param}to{date_param},a:all"
            f"&start={page}"
        )

        headers = {"User-Agent": "Mozilla/5.0"}
        resp = requests.get(url, headers=headers)
        soup = BeautifulSoup(resp.text, "html.parser")

        for title_tag in soup.select("a.news_tit"):
            titles.append(title_tag["title"])

        time.sleep(1.2)

    # 🔸 결과 저장
    results.append({
        "연예인 이름": name,
        "사건 날짜": date.strftime("%Y-%m-%d"),
        "기사 수": len(titles),
        "기사 제목 목록": "; ".join(titles)
    })

    print(f"{name} ({date.date()}) → {len(titles)}건 수집 완료")

# 🔹 저장
result_df = pd.DataFrame(results)
result_df.to_csv("연예인_뉴스기사제목_카운트결과.csv", index=False)

지민 (2022-04-25) → 0건 수집 완료
정국 (2024-01-06) → 0건 수집 완료
정원 (2021-11-18) → 0건 수집 완료
정원 (2021-11-19) → 0건 수집 완료


KeyboardInterrupt: 

In [12]:
df = pd.read_csv("C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/데이터/스캔들/ALL논란_Final.csv")
df["사건 날짜"] = pd.to_datetime(df["사건 날짜"])
name_date_df = df[["연예인 이름", "사건 날짜"]].drop_duplicates()

In [9]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
import time

# 함수 정의: div 개수로 기사 수 측정
def get_news_count_by_div(name, date_str, pages=40):
    query = urllib.parse.quote(name)
    date_param = date_str.replace(".", "")
    total_count = 0

    for page in range(1, pages * 10, 10):
        url = (
            f"https://search.naver.com/search.naver"
            f"?where=news&query={query}&sort=0"
            f"&ds={date_str}&de={date_str}"
            f"&nso=so:r,p:from{date_param}to{date_param},a:all"
            f"&start={page}"
        )

        headers = {"User-Agent": "Mozilla/5.0"}
        resp = requests.get(url, headers=headers)
        soup = BeautifulSoup(resp.text, "html.parser")

        divs = soup.select("div.iyO99lP86ixD0iM_4cb8")
        if not divs:
            break  # 뉴스 없으면 종료
        total_count += len(divs)
        time.sleep(1.2)  # 요청 간 딜레이

    return total_count


In [10]:
from datetime import datetime

# 사건 날짜 형식을 문자열로 변환
name_date_df["사건 날짜"] = pd.to_datetime(name_date_df["사건 날짜"])
name_date_df["사건 날짜 문자열"] = name_date_df["사건 날짜"].dt.strftime("%Y.%m.%d")

# 상위 5개만 먼저 테스트로 실행 (전체 실행 시 시간 오래 걸릴 수 있음)
sample_df = name_date_df.head(5).copy()

# 기사 수 수집
counts = []
for _, row in sample_df.iterrows():
    name = row["연예인 이름"]
    date_str = row["사건 날짜 문자열"]
    try:
        count = get_news_count_by_div(name, date_str)
        print(f"{name} / {date_str} → {count}건")
    except Exception as e:
        print(f"오류: {name} / {date_str} → {e}")
        count = None
    counts.append(count)

# 결과 추가
sample_df["네이버 뉴스 기사 수(div 기반)"] = counts
sample_df

지민 / 2022.04.25 → 0건
정국 / 2024.01.06 → 0건
정원 / 2021.11.18 → 0건
정원 / 2021.11.19 → 0건
제이 / 2023.01.10 → 0건


,연예인 이름,사건 날짜,사건 날짜 문자열,네이버 뉴스 기사 수(div 기반)
0,지민,2022-04-25,2022.04.25,0
1,정국,2024-01-06,2024.01.06,0
2,정원,2021-11-18,2021.11.18,0
3,정원,2021-11-19,2021.11.19,0
4,제이,2023-01-10,2023.01.10,0


In [13]:
# 필요한 코드 전체 묶음입니다.
# (이전에 보낸 get_news_count_by_div() 함수 포함)

# ▶ 연예인 CSV 불러오기
import pandas as pd
df["사건 날짜"] = pd.to_datetime(df["사건 날짜"])
df = df[["연예인 이름", "사건 날짜"]].drop_duplicates()
df["사건 날짜 문자열"] = df["사건 날짜"].dt.strftime("%Y.%m.%d")

# ▶ 기사 수 수집 실행
from time import sleep

results = []
for _, row in df.iterrows():
    name = row["연예인 이름"]
    date_str = row["사건 날짜 문자열"]
    try:
        count = get_news_count_by_div(name, date_str)
        print(f"{name} / {date_str} → {count}건")
    except Exception as e:
        print(f"오류: {name} / {date_str} → {e}")
        count = None
    results.append({
        "연예인 이름": name,
        "사건 날짜": row["사건 날짜"],
        "기사 수(div 기반)": count
    })
    sleep(1)

# ▶ 결과 저장
pd.DataFrame(results)
results


지민 / 2022.04.25 → 0건
정국 / 2024.01.06 → 0건
정원 / 2021.11.18 → 0건
정원 / 2021.11.19 → 0건
제이 / 2023.01.10 → 0건
니키 / 2024.02.29 → 0건
김채원 / 2022.05.13 → 0건
김채원 / 2023.02.10 → 0건
김채원 / 2023.09.05 → 0건
김채원 / 2022.10.19 → 0건
김채원 / 2024.04.03 → 0건
김채원 / 2024.04.13 → 0건


KeyboardInterrupt: 

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import urllib.parse

def scroll_to_bottom(driver, pause_time=1.5):
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # 스크롤 맨 아래로
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause_time)
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # 더 이상 스크롤 불가
        last_height = new_height

def get_news_count_scroll(name, date_str):
    query = urllib.parse.quote(name)
    date_param = date_str.replace(".", "")

    url = (
        f"https://search.naver.com/search.naver"
        f"?where=news&query={query}&sort=0"
        f"&ds={date_str}&de={date_str}"
        f"&nso=so:r,p:from{date_param}to{date_param},a:all"
    )

    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=options)

    driver.get(url)
    scroll_to_bottom(driver)

    # 기사 개별 div 수집
    try:
        news_items = driver.find_elements(By.CSS_SELECTOR, "div.group_news > ul > li")
        count = len(news_items)
    except:
        count = 0

    driver.quit()
    return count


In [15]:
get_news_count_scroll("백현", "2024.05.17")
# 결과: 예를 들어 37


0

In [ ]:
##########################

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_1samp

summary_rows = []

for day in range(-10, 11):
    day_df = abnormal[abnormal['event_window_day'] == day]
    ar_values = day_df['비정상 수익률']

    if len(ar_values) == 0:
        continue

    aar = ar_values.mean()
    std = ar_values.std(ddof=1)
    n = len(ar_values)

    # t검정
    t_stat, p_val = ttest_1samp(ar_values, 0)

    # CAAR = AAR 누적합 (여기선 누적은 표 생성 이후 별도 처리)
    # 음의 수익률 비율
    neg_ratio = (ar_values < 0).mean() * 100

    summary_rows.append({
        'Event day': f"AR({day})",
        'AAR(%)': round(aar * 100, 2),
        'T-Statistic': round(t_stat, 5),
        'P-value': round(p_val, 3),
        'Negative AR%': round(neg_ratio, 2)
    })

# DataFrame 생성
summary_df = pd.DataFrame(summary_rows)

# CAAR 누적합 추가
summary_df['CAAR(%)'] = summary_df['AAR(%)'].cumsum()

# 열 순서 정리
summary_df = summary_df[['Event day', 'AAR(%)', 'T-Statistic', 'P-value', 'CAAR(%)', 'Negative AR%']]
summary_df.columns = ['Event day', 'Average Abnormal Return(%)', 'T-Statistic', 'P-value',
                      'Cumulative Average Abnormal Return(%)', '% of -Abnormal Returns']

# 결과 확인
print(summary_df)
# summary_df.to_csv("event_study_summary_table.csv", index=False)
